In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb

In [2]:
train = pd.read_csv('./train_1.csv').fillna(0)
key = pd.read_csv('./key_1.csv')

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [4]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

In [41]:
def get_median(row, last_n_days):
    return np.median(list(row.values)[-1*last_n_days : ])

In [46]:
def get_mean(row, last_n_days):
    return np.mean(list(row.values)[-1*last_n_days : ])

In [53]:
def traffic_stats():
        
    train_feats["median_7_days"] = train.apply(get_median, last_n_days = 7, axis = 1)
    train_feats["mean_7_days"] = train.apply(get_mean, last_n_days = 7, axis = 1)    

    train_feats["median_14_days"] = train.apply(get_median, last_n_days = 14, axis = 1)
    train_feats["mean_14_days"] = train.apply(get_mean, last_n_days = 14, axis = 1)    

    train_feats["median_21_days"] = train.apply(get_median, last_n_days = 21, axis = 1)
    train_feats["mean_21_days"] = train.apply(get_mean, last_n_days = 21, axis = 1)    

    train_feats["median_30_days"] = train.apply(get_median, last_n_days = 30, axis = 1)
    train_feats["mean_30_days"] = train.apply(get_mean, last_n_days = 30, axis = 1)    

    train_feats["median_60_days"] = train.apply(get_median, last_n_days = 60, axis = 1)
    train_feats["mean_60_days"] = train.apply(get_mean, last_n_days = 60, axis = 1)        
    

In [74]:
train_feats = pd.DataFrame()
train_feats["Page"] = train["Page"]

traffic_stats()

train_feats.to_csv("train_feats.csv", index = False)

In [86]:
key["trunc_page"] = key.apply(lambda x: x["Page"][:-11], axis = 1)
key.to_csv("key_trunc.csv", index = False)

In [87]:
train_feats = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_merged = pd.merge(train_feats, key, left_on="Page", right_on="trunc_page", how="inner")
train_merged.drop("Page_x", inplace=True, axis = 1)
train_merged.drop("Page_y", inplace=True, axis = 1)

train_merged.to_csv("train_merged.csv", index = False)

In [ ]:
def run_xgb(x_train, x_test, x_label):

    # x_train = pd.concat([pos_train, neg_train]) #Concat positive and negative
    # y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist() #Putting in 1 and 0

    # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.05
    params['max_depth'] = 6
    params['silent'] = 1

    d_train = xgb.DMatrix(x_train, label=x_label)
    d_test = xgb.DMatrix(x_test)

    watchlist = [(d_train, 'train')]

    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=50, verbose_eval=50)

    p_test = bst.predict(d_test)

    # xgb.plot_importance(bst)
    # pyplot.show()

    return p_test